In [2]:
import pandas as pd

In [12]:
# Importing Libraries
import os
from dotenv import load_dotenv
load_dotenv()

from fredapi import Fred

fred_api_key = os.getenv('API_KEY')

In [13]:
pd.set_option('display.max_colwidth', None)

In [14]:
# Creating the Fred Object
fred = Fred(fred_api_key)

In [18]:
cpi_df = fred.search('Consumer Price Index for All Urban Consumers: All Items in U.S. City Average',filter=('frequency', 'Monthly'))
cpi_baseYear = cpi_df.query('seasonal_adjustment== "Seasonally Adjusted" and units == "Index 1982-1984=100" ')

In [23]:
cpi_df['notes']

series id
CPIAUCSL         The Consumer Price Index for All Urban Consumers: All Items (CPIAUCSL) is a price index of a basket of goods and services paid by urban consumers. Percent changes in the price index measure the inflation rate between any two time periods. The most common inflation metric is the percent change from one year ago. It can also represent the buying habits of urban consumers. This particular index includes roughly 88 percent of the total population, accounting for wage earners, clerical workers, technical workers, self-employed, short-term workers, unemployed, retirees, and those not in the labor force.  The CPIs are based on prices for food, clothing, shelter, and fuels; transportation fares; service fees (e.g., water and sewer service); and sales taxes. Prices are collected monthly from about 4,000 housing units and approximately 26,000 retail establishments across 87 urban areas. To calculate the index, price changes are averaged with weights representing their 

In [20]:
cpi_baseYear = cpi_df.query('seasonal_adjustment== "Seasonally Adjusted" and units == "Index 1982-1984=100" ')
core_cpi = cpi_baseYear.head(4)

In [21]:
# Adding all the series in one dataFrame
all_results = []

for sID in core_cpi.index:
    results = fred.get_series(sID)
    results = results.to_frame(name=sID)
    all_results.append(results)
df_core_cpi = pd.concat(all_results, axis=1).dropna()

In [22]:
df_core_cpi

,CPIAUCSL,CPILFESL,CWSR0000SA0,CUSR0000SA0L2
1957-01-01,27.670,28.500,27.820,29.300
1957-02-01,27.800,28.600,27.960,29.400
1957-03-01,27.860,28.700,28.020,29.500
1957-04-01,27.930,28.800,28.090,29.600
1957-05-01,28.000,28.800,28.160,29.600
...,...,...,...,...
2024-04-01,313.207,317.622,307.496,283.849
2024-05-01,313.225,318.140,307.330,283.228
2024-06-01,313.049,318.346,306.946,282.703
2024-07-01,313.534,318.872,307.419,282.779


In [76]:
# Renaming columns
df_core_cpi.rename(columns={
    'CPIAUCSL': 'CPI_All_Items',
    'CPILFESL': 'CPI_Less_Food_Energy'
}, inplace=True)

In [79]:
df_core_cpi.to_csv("data.csv")